### Reference
   * [Dataflow programming wiki](https://en.wikipedia.org/wiki/Dataflow_programming) : dataflow 프로그래밍에 대해 자세히 설명되어 있음
   * [밑바닥부터 시작하는 딥러닝 - chapter 4](https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch04/two_layer_net.py) : 참고한 코드
   
   * [Deep Learning From Scratch ](http://www.deepideas.net/deep-learning-from-scratch-i-computational-graphs/)

In [141]:
%matplotlib inline
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

## Dataflow Programming

데이터플로우 프로그래밍은 **하나의 프로그래밍 패턴**이다. 우리가 일반적으로 하는 대부분의 프로그래밍의 형태는 Control Flow의 순서에 따라 코드가 명세되어 있다. 즉, 모든 코드들이 작업(operation)의 Order를 중심으로 서술되어 있다. 예를 들어, 2개의 input(x,y)을 받아서 아래와 같은 연산을 하는 녀석을 생각해 보자. 

$$
z = 3*(x+y)^2
$$

In [142]:
def calculate(x, y):
    t1 = x + y       # (1)
    t2 = (x + y)**2  # (2)
    z = 3 * t2       # (3)
    return z

우리는 Z를 구하기 위한 operation의 Sequence를 중심으로 적는다. 이렇게 하는 방식을 **Sequential, 혹은 Control Flow의 방식으로 코드를 짠다**라고 한다. 이렇게 프로그래밍하면 어떤 아쉬움이 존재할까? 

우리가 역전파를 구현한다고 생각해보자. 그러면 어떤 식으로 gradient를 구해야 할까? 문제는 바로 z에 연결된 이전 계산이 무엇인지 판단하기 어렵다. 핵심은 Data가 어떤 operation의 순으로 연결되어 있는지를 알아야 한다는 것이다.

Auto Diffferentiation, 즉 자동 역전파 코드를 구현하기 위해서는, 데이터 간 연결을 명세할 필요가 있다는 것이다. 그래서 텐서플로우는 각 데이터가 어떤 순서대로 움직이는 지를 명세하는 Data Flow 방식으로 코드를 짰다. 


![](https://www.tensorflow.org/images/tensors_flowing.gif)


이렇게 코드를 짰을 때 가지는 이점들은 아래와 같다.

1. Deep Flexibility

2. True Portability

3. Connect Research and Production

4. Auto-Differentiation

5. Language Options

6. Maximize Performance

## 넘파이로 2층짜리 신경망 클래스를 짜보자

### (1) Dataflow가 아닌, 일반적 방식 (Control Flow)으로 코드를 구현한 경우

In [143]:
################
# Helper Method for Neural Network
# 
################
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

def sigmoid(x):
    return 1/(1+np.exp(-x))

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, 
                 output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(x, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        batch_size = x.shape[0]
        accuracy = np.sum(y == t) / batch_size
        return accuracy
        
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        # Core Problem!!!
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    

weight 갱신을 하기 위해서, 총 4번의 Numerical_gradient가 필요한데
여러 번의 feed-forward를 거쳐야 하게 됨

-> 여기서 매우 큰 연산 비용(computational Cost)가 발생

그렇기 때문에 우리는 일반적으로 오차역전파법으로 코드를 구현하게 되고, 오차 역전파법을 하기 위해서는, 기본적으로는 계산 그래프(Computational Graph)가 적용하기 유용하다.

### (2) DataFlow의 형태로 코드를 구현한 경우

> 먼저 DataFlow, 즉 Graph로 코드를 짜기 전에 Graph의 구성부터 깊게 따져보자.

우리가 주로 다루는 계산 그래프(Computational Graph)는 Node와 Edge로 구성된 유향 그래프(directed graph)이다. 보통 Node를 Operation의 역할을 하고, Edge에는 Tensor가 들어가게 된다. 

![](../../misc/tensorflow-basis/computational_graph.png)

In [144]:
class Graph:
    """Represents a computational graph
    """

    def __init__(self):
        """Construct Graph"""
        self.operations = [] # Node가 들어가는 부분
        self.placeholders = [] # Feeding할 때 들어가는 placeholder
        self.variables = [] # 함수

    def as_default(self):
        global _default_graph
        _default_graph = self


class Operation:
    """Represents a graph node that performs a computation.
    An `Operation` is a node in a `Graph` that takes zero or
    more objects as input, and produces zero or more objects
    as output.
    """

    def __init__(self, input_nodes=[]):
        """Construct Operation
        """
        self.input_nodes = input_nodes

        # Initialize list of consumers (i.e. nodes that receive this operation's output as input)
        self.consumers = []

        # Append this operation to the list of consumers of all input nodes
        for input_node in input_nodes:
            input_node.consumers.append(self)

        # Append this operation to the list of operations in the currently active default graph
        _default_graph.operations.append(self)

    def compute(self):
        """Computes the output of this operation.
        "" Must be implemented by the particular operation.
        """
        pass


class placeholder:
    """Represents a placeholder node that has to be provided with a value
       when computing the output of a computational graph
    """

    def __init__(self):
        """Construct placeholder
        """
        self.consumers = []

        # Append this placeholder to the list of placeholders in the currently active default graph
        _default_graph.placeholders.append(self)


class Variable:
    """Represents a variable (i.e. an intrinsic, changeable parameter of a computational graph).
    """

    def __init__(self, initial_value=None):
        """Construct Variable
        Args:
          initial_value: The initial value of this variable
        """
        self.value = initial_value
        self.consumers = []

        # Append this variable to the list of variables in the currently active default graph
        _default_graph.variables.append(self)

In [145]:

class add(Operation):
    """Returns x + y element-wise.
    """

    def __init__(self, x, y):
        """Construct add
        Args:
          x: First summand node
          y: Second summand node
        """
        super().__init__([x, y])

    def compute(self, x_value, y_value):
        """Compute the output of the add operation
        Args:
          x_value: First summand value
          y_value: Second summand value
        """
        self.inputs = [x_value, y_value]
        return x_value + y_value


class matmul(Operation):
    """Multiplies matrix a by matrix b, producing a * b.
    """

    def __init__(self, a, b):
        """Construct matmul
        Args:
          a: First matrix
          b: Second matrix
        """
        super().__init__([a, b])

    def compute(self, a_value, b_value):
        """Compute the output of the matmul operation
        Args:
          a_value: First matrix value
          b_value: Second matrix value
        """
        self.inputs = [a_value, b_value]
        return a_value.dot(b_value)

In [146]:
class Session:
    """Represents a particular execution of a computational graph.
    """

    def run(self, operation, feed_dict={}):
        """Computes the output of an operation
        Args:
          operation: The operation whose output we'd like to compute.
          feed_dict: A dictionary that maps placeholders to values for this session
        """

        # Perform a post-order traversal of the graph to bring the nodes into the right order
        nodes_postorder = traverse_postorder(operation)

        # Iterate all nodes to determine their value
        for node in nodes_postorder:

            if type(node) == placeholder:
                # Set the node value to the placeholder value from feed_dict
                node.output = feed_dict[node]
            elif type(node) == Variable:
                # Set the node value to the variable's value attribute
                node.output = node.value
            else:  # Operation
                # Get the input values for this operation from node_values
                node.inputs = [input_node.output for input_node in node.input_nodes]

                # Compute the output of this operation
                node.output = node.compute(*node.inputs)

            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)

        # Return the requested node value
        return operation.output


def traverse_postorder(operation):
    """Performs a post-order traversal, returning a list of nodes
    in the order in which they have to be computed
    Args:
       operation: The operation to start traversal at
    """

    nodes_postorder = []

    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

#### 이것을 가지고 코드를 돌려보면 아래와같다.

In [147]:
# Create a new graph
Graph().as_default()

# Create variables
A = Variable([[1, 0], [0, -1]])
b = Variable([1, 1])

# Create placeholder
x = placeholder()

# Create hidden node y
y = matmul(A, x)

# Create output node z
z = add(y, b)

In [148]:
sess = Session()

In [149]:
sess.run(z, feed_dict={x:[1,2]})

array([ 2, -1])

*Backpropagation까지 전반적인 코드를 전부 상세히 다루는 것은 생각보다 너무 빡세서, 일단 Graph와 Session이 어떤 식으로 구성되어 있는지를 코드로만 알려줌*


**TO-DO LIST -> 역전파와 순전파까지 코드를 다루자**

이렇게 코드를 짤 경우, 우리는 각 Operation이 어느 단계의 Operation에 묶여 있는지를 알 수 있다. 왜냐하면, 각 Operation은 `consumers`라는 인자 값으로, 이전 Operation이 무엇이었는지 명세하고 있기 때문이다. 즉, 우리는 역전파를 할 때, Error Signal을 전파하기 가능한 구조로  짜져 있다. 

<hr>

Copyright(c) 2019 by Public AI. All rights reserved.<br>
Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/01/24
<hr>